# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [38]:
# import libraries
import pandas as pd
import sqlalchemy

from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize

from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.multioutput import MultiOutputClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.pipeline import Pipeline


In [20]:
# load data from database
engine = sqlalchemy.create_engine('sqlite:///etl_data.db')
df = pd.read_sql_table('disaster_response', engine)

# X = df[['message']].values[:, 0]
# Y = df[df.columns[4:]].values
X = df["message"]
Y = df.drop(['message', 'genre', 'id', 'original'], axis = 1)

### 2. Write a tokenization function to process your text data

In [21]:
def tokenize(message):
    tokens = word_tokenize(message)
    lemmatizer = WordNetLemmatizer()
    clean_tokens = []
    for token in tokens:
        clean_token = lemmatizer.lemmatize(token).lower().strip()
        clean_tokens.append(clean_token)
        
    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [43]:
pipeline = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(RandomForestClassifier(n_estimators = 20)))
        ])

In [44]:
pipeline.get_params()

{'memory': None,
 'steps': [('vect',
   CountVectorizer(analyzer='word', binary=False, decode_error='strict',
           dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
           lowercase=True, max_df=1.0, max_features=None, min_df=1,
           ngram_range=(1, 1), preprocessor=None, stop_words=None,
           strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
           tokenizer=<function tokenize at 0x123951620>, vocabulary=None)),
  ('tfidf',
   TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)),
  ('clf',
   MultiOutputClassifier(estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
               max_depth=None, max_features='auto', max_leaf_nodes=None,
               min_impurity_decrease=0.0, min_impurity_split=None,
               min_samples_leaf=1, min_samples_split=2,
               min_weight_fraction_leaf=0.0, n_estimators=20, n_jobs=None,
               oob_score=False, random_state=None,

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [45]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 0.2, random_state = 7)

In [46]:
pipeline.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip..._score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=None))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [47]:
y_pred = pipeline.predict(X_test)
for i, col in enumerate(y_test):
    print(classification_report(y_test[col], y_pred[:, i]))

              precision    recall  f1-score   support

           0       0.70      0.30      0.42      1245
           1       0.81      0.96      0.88      3946
           2       0.29      0.16      0.20        32

   micro avg       0.80      0.80      0.80      5223
   macro avg       0.60      0.47      0.50      5223
weighted avg       0.78      0.80      0.76      5223

              precision    recall  f1-score   support

           0       0.89      0.99      0.94      4357
           1       0.86      0.41      0.55       866

   micro avg       0.89      0.89      0.89      5223
   macro avg       0.88      0.70      0.75      5223
weighted avg       0.89      0.89      0.87      5223

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      5199
           1       0.00      0.00      0.00        24

   micro avg       1.00      1.00      1.00      5223
   macro avg       0.50      0.50      0.50      5223
weighted avg       0

              precision    recall  f1-score   support

           0       0.86      0.99      0.92      4206
           1       0.84      0.31      0.46      1017

   micro avg       0.85      0.85      0.85      5223
   macro avg       0.85      0.65      0.69      5223
weighted avg       0.85      0.85      0.83      5223



/Users/Agata/.pyenv/versions/3.7.0/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 6. Improve your model
Use grid search to find better parameters. 

In [55]:
parameters = {'tfidf__use_idf': (True, False), 
              'clf__estimator__n_estimators': [20, 50], 
              'clf__estimator__min_samples_split': [2, 4]} 

cv = GridSearchCV(pipeline, param_grid = parameters, cv = 3,)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [56]:
cv.fit(X_train, y_train)
perf_report(cv, X_test, y_test)

KeyboardInterrupt: 


KeyboardInterrupt



### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [ ]:
pipeline_2 = Pipeline([
    ('vect', CountVectorizer()),
    ('best', TruncatedSVD()),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(AdaBoostClassifier()))
])

In [ ]:
pipeline_2.get_params()

In [ ]:
pipeline_2.fit(X_train, y_train)

In [ ]:
perf_report(pipeline_2, X_test, y_test)

In [ ]:
parameters_2 = {'tfidf__use_idf': (True, False), 
              'clf__estimator__n_estimators': [20, 50], 
              'clf__estimator__min_samples_split': [2, 4]} 

cv_2 = GridSearchCV(pipeline_2, param_grid = parameters_2, cv = 3,)

In [ ]:
cv_2.fit(X_train, y_train)
perf_report(cv_2, X_test, y_test)

### 9. Export your model as a pickle file

In [ ]:
with open('model.pkl', 'wb') as f:
    pickle.dump(cv_2, f)

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.